In [1]:
# Main setup for network



import gym
import retro
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
from retro import data
import random
from collections import deque


class SMBAI(nn.Module):
    def __init__(self):
        super(SMBAI, self).__init__()
        self.Conv1 = nn.Conv2d(3, 9, 5, groups=3)
        self.Conv2 = nn.Conv2d(9, 45, 5)
        self.Conv3 = nn.Conv2d(45, 90, 5)
        self.MaxPool = nn.MaxPool2d(16)
        self.Lin1 = nn.Linear(16380, 5000)
        self.Lin2 = nn.Linear(5000, 2000)
        self.Lin3 = nn.Linear(2000, 500)
        self.Lin4 = nn.Linear(500, 7)
        
    def forward(self, x):
        x = x.unsqueeze(0)
        x = self.Conv1(x)
        x = self.Conv2(x)
        x = self.Conv3(x)
        x = self.MaxPool(x)
        x = x.view(-1, 16380)
        x = self.Lin1(x)
        x = self.Lin2(x)
        x = self.Lin3(x)
        x = self.Lin4(x)
        x = x.squeeze(0)
        return x
def ChooseButton(x, epsilon, test=True):
    if test == True:
        y = [0, 0, 0, 0, 0, 0, 0, 0, 0]
        maxval, index = torch.max(x, 0)
        if index == 0:
            y = [0, 0, 0, 0, 0, 0, 0, 1, 0]
        if index == 1:
            y = [1, 0, 0, 0, 0, 0, 0, 1, 0]
        if index == 2:
            y = [1, 0, 0, 0, 0, 0, 0, 1, 1]
        if index == 3:
            y = [0, 0, 0, 0, 0, 0, 0, 1, 1]
        if index == 4:
            y = [0, 0, 0, 0, 0, 0, 1, 0, 0]
        if index == 5:
            y = [0, 0, 0, 0, 0, 0, 1, 0, 1]
        if index == 6:
            y = [1, 0, 0, 0, 0, 0, 1, 0, 1]
        return y, epsilon
    else:
        if random.uniform(0, 1) <= epsilon: #Use random button with rightward bias
            y = [0, 0, 0, 0, 0, 0, 0, 0, 0]
            randb = random.randint(0, 21)
            if randb == 0 or randb == 1 or randb == 2 or randb == 3:
                y = [0, 0, 0, 0, 0, 0, 0, 1, 0]
            if randb == 4 or randb == 5 or randb == 6 or randb == 7:
                y = [1, 0, 0, 0, 0, 0, 0, 1, 0]
            if randb == 8 or randb == 9 or randb == 10 or randb == 11:
                y = [1, 0, 0, 0, 0, 0, 0, 1, 1]
            if randb == 12 or randb == 13 or randb == 14 or randb == 15:
                y = [0, 0, 0, 0, 0, 0, 0, 1, 1]
            if randb == 16 or randb == 17:
                y = [0, 0, 0, 0, 0, 0, 1, 0, 0]
            if randb == 18 or randb == 19:
                y = [0, 0, 0, 0, 0, 0, 1, 0, 1]
            if randb == 20 or randb == 21:
                y = [1, 0, 0, 0, 0, 0, 1, 0, 1]
        else: #Use best Q-val
            y = [0, 0, 0, 0, 0, 0, 0, 0, 0]
            maxval, index = torch.max(x, 0)
            if index == 0:
                y = [0, 0, 0, 0, 0, 0, 0, 1, 0]
            if index == 1:
                y = [1, 0, 0, 0, 0, 0, 0, 1, 0]
            if index == 2:
                y = [1, 0, 0, 0, 0, 0, 0, 1, 1]
            if index == 3:
                y = [0, 0, 0, 0, 0, 0, 0, 1, 1]
            if index == 4:
                y = [0, 0, 0, 0, 0, 0, 1, 0, 0]
            if index == 5:
                y = [0, 0, 0, 0, 0, 0, 1, 0, 1]
            if index == 6:
                y = [1, 0, 0, 0, 0, 0, 1, 0, 1]
        global epsilon_min
        global epsilon_decay
        if epsilon > epsilon_min:
            epsilon *= epsilon_decay
        return y, epsilon
    
#def ChooseButton(x, epsilon, test=True):
    #if test == True:
        #y = [0, 0, 0, 0, 0, 0, 0, 0, 0]
        #maxval = torch.max(x)
        #for i, z in enumerate(x):
         #   if maxval == z:
          #      y[i] = 1
        #return y, epsilon
   # else:
       # if random.uniform(0, 1) <= epsilon: #Use random button
       #     y = [0, 0, 0, 0, 0, 0, 0, 0, 0]
       #     randb = random.randint(0, 8)
        #    y[randb] = 1
       # else: #Use best Q-val
          #  y = [0, 0, 0, 0, 0, 0, 0, 0, 0]
          #  maxval = torch.max(x)
          #  for i, z in enumerate(x):
          #      if maxval == z:
          #          y[i] = 1
        #global epsilon_min
        #global epsilon_decay
        #if epsilon > epsilon_min:
            #epsilon *= epsilon_decay
        #return y, epsilon
def Qprocess():
    Qbatch = random.sample(Qlog, batch_size)
    totalloss = 0
    for state, action, reward, nextstate, done in Qbatch:
        #state = torch.reshape(state, (3, 224, 240))
        #state = state.cuda()
        target = reward
        val = max(AI(nextstate))
        target = (reward + gamma * val)
        target_actual = AI(state).data
        target_actual[action] = target
        OptimF1.zero_grad()
        out = AI(state)
        loss = LossF1(out, target_actual)
        with torch.no_grad():
            totalloss = totalloss + loss.item()
        loss.backward()
        OptimF1.step()
        loss = 0
    return totalloss

In [2]:
# Training Loop

try:
    env
except NameError:
    env = retro.make(game='SuperMarioBros-Nes', state='Level1-1')
else:
    env.close()
    env = retro.make(game='SuperMarioBros-Nes', state='Level1-1')
epsilon = 1.00
epsilon_min = .08
epsilon_decay = .995
gamma = .03
procmultiplier = 1
lives = 2
Qlog = deque(maxlen=1200)
AI = SMBAI()
AI = AI.cuda()
LossF1 = nn.MSELoss()
OptimF1 = optim.Adam(AI.parameters(), lr=.00001)
frameval = 600
frames = frameval
epochs = 25
batch_size = 5
totalloss = 0
megatotalloss = 0
totalrew = 0
maxtotalrew = 0
for x in range(epochs):
    megatotalloss = 0
    totalrew = 0
    done = False
    obs = env.reset() # 224x240x3
    obs = torch.from_numpy(obs)
    obs = obs.view(3, 224, 240)
    obs = torch.tensor(obs, dtype = torch.float32, device = 'cuda')
    output = AI(obs)
    button, epsilon = ChooseButton(output, epsilon, test=False)
    frames = frameval
    
    
    
    # PRIORITY 1: Reduce reward with time
    #
    # 
    #
    #
    
    
    
    while not done: 
        frames = frames - 1
        obs2 = obs
        #obs, rew, done, info = env.step([0, 0, 0, 0, 0, 0, 0, 1, 0])
        maxlives = lives
        obs, rew, done, info = env.step(button)
        lives = info['lives']
        print(info)
        #print(lives)
        if lives < maxlives:
            rew = rew - 10
            done = True
        #print(rew)
        rew = rew - .1
        totalrew += rew
        #env.render()
        obs = torch.from_numpy(obs)
        obs = obs.view(3, 224, 240)
        obs = torch.tensor(obs, dtype = torch.float32, device = 'cuda')
        Qlog.append((obs2, button, rew, obs, done))
        #reward = rew if not done else rew-100
        #pvals = torch.LongTensor(pvals, device = 'cuda')
        output = AI(obs)
        button, epsilon = ChooseButton(output, epsilon, test=False)
        #print(button)
        print(button)
        print(rew)
        if len(Qlog) > batch_size:
            for alpha3357 in range(procmultiplier):
                totalloss = Qprocess()
                megatotalloss += totalloss
                print("Total loss this process: " + str(totalloss))
        #env.render()
        #if frames % 10 == 0:
            #output = AI(obs)
            #button, epsilon = ChooseButton(output, epsilon, test=False)
            #env.step([0, 0, 0, 0, 0, 0, 0, 0, 0])
        print('Proccessed ' + str(frameval - frames) + ' frames!')
        if frames == 0:
            done = True
        if done:
            print("episode: {}/{}, score: {}, e: {}".format(x, epochs, totalrew, epsilon))
    if totalrew <= maxtotalrew:
        maxtotalrew = totalrew
        device='cuda'



env.close()

{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 400, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 1]
-0.1
Proccessed 1 frames!
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 400, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
Proccessed 2 frames!
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 400, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 0, 1, 0]
-0.1
Proccessed 3 frames!
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 400, 'scrolling': 16, 'lives': 2, 'score': 0}
[1, 0, 0, 0, 0, 0, 0, 1, 0]
-0.1
Proccessed 4 frames!
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 400, 'scrolling': 16, 'lives': 2, 'score': 0}
[1, 0, 0, 0, 0, 0, 1, 0, 1]
-0.1
Proccessed 5 frames!
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 400, 'scrollin

tensor(-0.0186, device='cuda:0', grad_fn=<AddBackward>)
[0, 0, 0, 0, 0, 0, 0, 1, 0]
tensor([  2.7101,  -0.0136, -13.7639, -17.1456, -20.7305, -42.5028, -13.7636],
       device='cuda:0')
tensor([ -0.0186,  -0.0186, -13.7639, -17.1456, -20.7305, -42.5028, -13.7636],
       device='cuda:0', grad_fn=<PutBackward>)
tensor(-0.1185, device='cuda:0', grad_fn=<AddBackward>)
[0, 0, 0, 0, 0, 0, 0, 1, 0]
tensor([ -0.8481,  -0.5790, -13.9661, -17.6753, -22.1234, -42.8150, -13.7514],
       device='cuda:0')
tensor([ -0.1185,  -0.1185, -13.9661, -17.6753, -22.1234, -42.8150, -13.7514],
       device='cuda:0', grad_fn=<PutBackward>)
tensor(-0.1310, device='cuda:0', grad_fn=<AddBackward>)
[0, 0, 0, 0, 0, 0, 0, 1, 1]
tensor([ -3.8999,  -1.0106, -14.2199, -18.0330, -23.3862, -43.0572, -13.7276],
       device='cuda:0')
tensor([ -0.1310,  -0.1310, -14.2199, -18.0330, -23.3862, -43.0572, -13.7276],
       device='cuda:0', grad_fn=<PutBackward>)
tensor(-0.1590, device='cuda:0', grad_fn=<AddBackward>)
[0, 0

[1, 0, 0, 0, 0, 0, 0, 1, 1]
-0.1
tensor(-0.0963, device='cuda:0', grad_fn=<AddBackward>)
[0, 0, 0, 0, 0, 0, 0, 1, 1]
tensor([ -1.1053,   0.1434, -16.1221, -20.2173, -28.5078, -40.0483, -13.0696],
       device='cuda:0')
tensor([ -0.0963,  -0.0963, -16.1221, -20.2173, -28.5078, -40.0483, -13.0696],
       device='cuda:0', grad_fn=<PutBackward>)
tensor(-0.0845, device='cuda:0', grad_fn=<AddBackward>)
[1, 0, 0, 0, 0, 0, 0, 1, 0]
tensor([ -0.4507,   0.5026, -16.0763, -20.2548, -28.4386, -39.8049, -13.0065],
       device='cuda:0')
tensor([ -0.0845,  -0.0845, -16.0763, -20.2548, -28.4386, -39.8049, -13.0065],
       device='cuda:0', grad_fn=<PutBackward>)
tensor(-0.0733, device='cuda:0', grad_fn=<AddBackward>)
[0, 0, 0, 0, 0, 0, 1, 0, 0]
tensor([  0.5455,   0.8095, -16.2446, -20.4288, -28.2021, -39.7089, -12.8455],
       device='cuda:0')
tensor([ -0.0733,  -0.0733, -16.2446, -20.4288, -28.2021, -39.7089, -12.8455],
       device='cuda:0', grad_fn=<PutBackward>)
tensor(-0.0661, device='cuda

tensor(-0.1008, device='cuda:0', grad_fn=<AddBackward>)
[0, 0, 0, 0, 0, 0, 0, 1, 1]
tensor([ -0.3291,  -0.0844, -16.0175, -20.1303, -28.4214, -39.1147, -12.7486],
       device='cuda:0')
tensor([ -0.1008,  -0.1008, -16.0175, -20.1303, -28.4214, -39.1147, -12.7486],
       device='cuda:0', grad_fn=<PutBackward>)
Total loss this process: 1.457786325365305
Proccessed 19 frames!
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 399, 'scrolling': 16, 'lives': 2, 'score': 0}
[1, 0, 0, 0, 0, 0, 0, 1, 0]
-0.1
tensor(-0.1010, device='cuda:0', grad_fn=<AddBackward>)
[0, 0, 0, 0, 0, 0, 1, 0, 1]
tensor([ -0.5559,  -0.1468, -16.1430, -20.1680, -28.2964, -39.0577, -12.6463],
       device='cuda:0')
tensor([ -0.1010,  -0.1010, -16.1430, -20.1680, -28.2964, -39.0577, -12.6463],
       device='cuda:0', grad_fn=<PutBackward>)
tensor(-0.1136, device='cuda:0', grad_fn=<AddBackward>)
[0, 0, 0, 0, 0, 0, 0, 1, 0]
tensor([ -0.4885,  -0.4017, -16.0461, -19.9729, -28.4562, -39.139

tensor(-0.0991, device='cuda:0', grad_fn=<AddBackward>)
[0, 0, 0, 0, 0, 0, 0, 1, 1]
tensor([  0.0445,  -0.1016, -16.1414, -20.2336, -28.3521, -39.0616, -12.5999],
       device='cuda:0')
tensor([ -0.0991,  -0.0991, -16.1414, -20.2336, -28.3521, -39.0616, -12.5999],
       device='cuda:0', grad_fn=<PutBackward>)
tensor(-0.0981, device='cuda:0', grad_fn=<AddBackward>)
[1, 0, 0, 0, 0, 0, 0, 1, 0]
tensor([ -0.0386,  -0.3589, -16.0101, -20.2300, -28.3767, -39.1007, -12.6994],
       device='cuda:0')
tensor([ -0.0981,  -0.0981, -16.0101, -20.2300, -28.3767, -39.1007, -12.6994],
       device='cuda:0', grad_fn=<PutBackward>)
Total loss this process: 0.2677896926179528
Proccessed 24 frames!
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 399, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 0, 1, 1]
-0.1
tensor(-0.1017, device='cuda:0', grad_fn=<AddBackward>)
[0, 0, 0, 0, 0, 0, 0, 1, 1]
tensor([ -0.0328,  -0.1998, -16.1284, -20.1864, -28.3026, -39.02

tensor(-0.1008, device='cuda:0', grad_fn=<AddBackward>)
[0, 0, 0, 0, 0, 0, 0, 1, 1]
tensor([ -0.0897,  -0.0141, -15.9672, -19.9501, -28.3317, -38.9756, -12.6748],
       device='cuda:0')
tensor([ -0.1008,  -0.1008, -15.9672, -19.9501, -28.3317, -38.9756, -12.6748],
       device='cuda:0', grad_fn=<PutBackward>)
tensor(-0.1013, device='cuda:0', grad_fn=<AddBackward>)
[0, 0, 0, 0, 0, 0, 1, 0, 1]
tensor([ -0.0887,  -0.2062, -15.9811, -20.0453, -28.3513, -38.8574, -12.7109],
       device='cuda:0')
tensor([ -0.1013,  -0.1013, -15.9811, -20.0453, -28.3513, -38.8574, -12.7109],
       device='cuda:0', grad_fn=<PutBackward>)
tensor(-0.1015, device='cuda:0', grad_fn=<AddBackward>)
[1, 0, 0, 0, 0, 0, 0, 1, 0]
tensor([ -0.0280,  -0.2355, -16.0717, -20.1878, -28.2909, -39.0840, -12.6764],
       device='cuda:0')
tensor([ -0.1015,  -0.1015, -16.0717, -20.1878, -28.2909, -39.0840, -12.6764],
       device='cuda:0', grad_fn=<PutBackward>)
Total loss this process: 0.1110048396512866
Proccessed 29 fra

KeyboardInterrupt: 

In [27]:
#Testing loop for the env
import time
try:
    env
except NameError:
    env = retro.make(game='SuperMarioBros-Nes', state='Level1-1')
else:
    env.close()
    env = retro.make(game='SuperMarioBros-Nes', state='Level1-1')
frames = 600
epsilon = .08
epsilon_min = .08
epsilon_decay = .9995
AI = SMBAI()
AI = AI.cuda()
AI = torch.load("./SMBAI.backup5.pth")
#AI.load_state_dict(torch.load("./SMBAI.backup5.pth"))
env.reset()
button = [0, 0, 0, 0, 0, 0, 0, 0, 0]
for x in range(frames):
    #epsilon = 1.000
    obs, rew, done, info = env.step(button)
    rew = rew-.1
    obs = torch.from_numpy(obs)
    obs = obs.view(3, 224, 240)
    obs = torch.tensor(obs, dtype = torch.float32, device = 'cuda')
    #output = AI(obs)
    #button, epsilon = ChooseButton(output, epsilon, test=True)
    print(button)
    print(rew)
    print(info)
    env.render()
    time.sleep(.01)
    #if x % 10 == 0:
    output = AI(obs)
    button, epsilon = ChooseButton(output, epsilon, test=True)
env.render(close=True)
env.close()



[0, 0, 0, 0, 0, 0, 0, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 400, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 400, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 400, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 400, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 400, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 400, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'level

[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 398, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 398, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 398, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 397, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 397, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 397, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'level

[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 395, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 395, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 395, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 395, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 395, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 395, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'level

[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 393, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 393, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 393, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 393, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 393, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 393, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'level

[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 391, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 391, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 391, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 390, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 390, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 390, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'level

...

[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 386, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 386, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 386, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 386, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 386, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 386, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'level

[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 384, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 383, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 383, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 383, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 383, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 383, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'level

[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 381, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 381, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 381, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 381, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 381, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 381, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'level

[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 379, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 379, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 379, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 379, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 379, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 379, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'level

[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 376, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 376, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 376, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 376, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 376, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'levelLo': 0, 'xscrollHi': 0, 'levelHi': 0, 'coins': 0, 'xscrollLo': 0, 'time': 376, 'scrolling': 16, 'lives': 2, 'score': 0}
[0, 0, 0, 0, 0, 0, 1, 0, 0]
-0.1
{'level

#torch.load(AI.state_dict(), "./SMBAI.backup1.pth")
AI = SMBAI()
AI = AI.cuda()
AI.load_state_dict(torch.load("./SMBAI.backup1.pth"))
try:
    env
except NameError:
    env = retro.make(game='SuperMarioBros-Nes', state='Level1-1')
else:
    env.close()
    env = retro.make(game='SuperMarioBros-Nes', state='Level1-1')
done = False
obs = env.reset() # 224x240x3
obs = torch.from_numpy(obs)
obs = obs.view(3, 224, 240)
obs = torch.tensor(obs, dtype = torch.float32, device = 'cuda')
output = AI(obs)
button, epsilon = ChooseButton(output, epsilon, test=False)
frameval = 600
frames = frameval
while not done: # CLEAN UP LOOP; PRIORITY 2
    frames = frames - 1
    obs2 = obs
    obs, rew, done, info = env.step(button)
    env.render()
    obs = torch.from_numpy(obs)
    obs = obs.view(3, 224, 240)
    obs = torch.tensor(obs, dtype = torch.float32, device = 'cuda')
    Qlog.append((obs2, button, rew, obs, done))
    reward = rew if not done else rew-100
        #pvals = torch.LongTensor(pvals, device = 'cuda')
    output = AI(obs)
    button, epsilon = ChooseButton(output, epsilon, test=True)
    print(button)
    if len(Qlog) > batch_size:
        for x in range(procmultiplier):
            totalloss = Qprocess()
            print("Total loss this process: " + str(totalloss))
    #env.render()
    if frames % 4 == 0:
        #env.step([0, 0, 0, 0, 0, 0, 0, 0, 0])
        print('Proccessed ' + str(frameval - frames) + ' frames!')
        
    if frames == 0:
        done = True
    if done:
        print("episode: {}/{}, score: {}, e: {}".format(x, epochs, reward, epsilon))


NOTES SECTION

So, the main issue right now is that the env does not update the reward until the AI has moved right for about 50 frames. A randomly moving AI will never achieve any reward, and thus will never learn anything.

Option 1: Hardcode moving right for a bit to give the AI an example of a 'good action'

Option 2: Open up the RAM values and find a more accurate x-pos (if there is one...)

Option 3: Change random movements to favor rightwards movements (ex. actions include right, right-run, right-jump, right-run-jump, left)

Personally, I'm favoring option 3. Option 1 feels too hardcoded and not independent enough, and from my preliminary research there probably isn't a variable that would satisfy option 2.

In [ ]:
from nes_py.wrappers import BinarySpaceToDiscreteSpaceEnv
import gym_super_mario_bros
import time
from gym_super_mario_bros.actions import COMPLEX_MOVEMENT
try:
    env
except NameError:
    env = gym_super_mario_bros.make('SuperMarioBros-v0')
    env = BinarySpaceToDiscreteSpaceEnv(env, COMPLEX_MOVEMENT)
else:
    #env.close()
    env = gym_super_mario_bros.make('SuperMarioBros-v0')
    env = BinarySpaceToDiscreteSpaceEnv(env, COMPLEX_MOVEMENT)
done = True
print(env.action_space)
for step in range(100):
    if done:
        state = env.reset()
    state, reward, done, info = env.step(11)
    print(reward)
    print(info)
    env.render()
    time.sleep(.03)
print(env.action_space)
env.close()

In [15]:
import gym
import retro
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
from retro import data
import random
from collections import deque
import gym.utils.play.PlayPlot as PlayPlot

def callback(obs_t, obs_tp1, action, rew, done, info):
    return [rew,]
plotter = PlayPlot(callback, 30 * 5, ["reward"])
env = gym.make("Pong-v4")
play(env, callback=plotter.callback)

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


ModuleNotFoundError: No module named 'gym.utils.play.PlayPlot'; 'gym.utils.play' is not a package